In [1]:
ls Data

ACS_16_5YR_B19001_with_ann.csv  FCC_data.csv
ACS_16_5YR_B19013_with_ann.csv  FCC_deps.csv


In [2]:
import pandas as pd
import re

In [3]:
acs1 = pd.read_csv('Data/ACS_16_5YR_B19001_with_ann.csv')
acs2 = pd.read_csv('Data/ACS_16_5YR_B19013_with_ann.csv')
fcc = pd.read_csv('Data/FCC_data.csv').drop('Unnamed: 0', axis=1)

In [4]:
acs1[['GEO.id2']].iloc[1]

GEO.id2    480291101001
Name: 1, dtype: object

In [5]:
fcc[['blockgroup']].iloc[0]

blockgroup    480291620041
Name: 0, dtype: int64

In [6]:
acs1cols = acs1[0:1]
acs = acs1[1:]

In [7]:
colnames = {'GEO.id2':'blockgroup'}



df = acs.drop(columns=['GEO.id','GEO.display-label']).rename(columns=colnames)
acs1cols = acs1cols.drop(columns=['GEO.id','GEO.display-label']).rename(columns=colnames)

In [8]:
acs1cols.columns

Index(['blockgroup', 'HD01_VD01', 'HD02_VD01', 'HD01_VD02', 'HD02_VD02',
       'HD01_VD03', 'HD02_VD03', 'HD01_VD04', 'HD02_VD04', 'HD01_VD05',
       'HD02_VD05', 'HD01_VD06', 'HD02_VD06', 'HD01_VD07', 'HD02_VD07',
       'HD01_VD08', 'HD02_VD08', 'HD01_VD09', 'HD02_VD09', 'HD01_VD10',
       'HD02_VD10', 'HD01_VD11', 'HD02_VD11', 'HD01_VD12', 'HD02_VD12',
       'HD01_VD13', 'HD02_VD13', 'HD01_VD14', 'HD02_VD14', 'HD01_VD15',
       'HD02_VD15', 'HD01_VD16', 'HD02_VD16', 'HD01_VD17', 'HD02_VD17'],
      dtype='object')

All HD02 Columns are the margin of error, so we can drop those

In [9]:
# Drops any cols that aren't HD01s, but keeps blockgroup
df =df[['blockgroup']+[col for col in df.columns[1:] if int(col[2:4]) == 1]]
acs1cols = acs1cols[['blockgroup']+[col for col in df.columns[1:] if int(col[2:4]) == 1]]

Now Let's rename the column names to something a little more human readable

In [10]:
columnnames = {'HD01_VD01':'pop_tot', 'HD01_VD02':'lessthan_10k','HD01_VD17':'greaterthan_200k'}



This is what the string representation of each column is. 

In [11]:
str(acs1cols['HD01_VD03'])

'0    Estimate; Total: - $10,000 to $14,999\nName: HD01_VD03, dtype: object'

Let's write a regular expression that extracts the Dollar amounts of those columns that follow this format

In [12]:
v = re.findall(r'\$(\d+),000 to \$(\d+)',str(acs1cols['HD01_VD03']))
v

[('10', '14')]

Now we can turn those two numbers into a string that represent the dollar amount

In [13]:
f'{v[0][0]}k-{v[0][1]}k'

'10k-14k'

Lets get the numbers of all the columns that follow that pattern and save it to colnums

In [14]:
colnums = {col:re.findall(r'\$(\d+),000 to \$(\d+)', str(acs1cols[col])) for col in acs1cols.columns[3:-1] }
colnums

{'HD01_VD03': [('10', '14')],
 'HD01_VD04': [('15', '19')],
 'HD01_VD05': [('20', '24')],
 'HD01_VD06': [('25', '29')],
 'HD01_VD07': [('30', '34')],
 'HD01_VD08': [('35', '39')],
 'HD01_VD09': [('40', '44')],
 'HD01_VD10': [('45', '49')],
 'HD01_VD11': [('50', '59')],
 'HD01_VD12': [('60', '74')],
 'HD01_VD13': [('75', '99')],
 'HD01_VD14': [('100', '124')],
 'HD01_VD15': [('125', '149')],
 'HD01_VD16': [('150', '199')]}

Now apply that string transformation to each of those numbers

In [15]:
for key in colnums:
    columnnames[key] = f'{colnums[key][0][0]}k-{colnums[key][0][1]}k'

In [16]:
columnnames

{'HD01_VD01': 'pop_tot',
 'HD01_VD02': 'lessthan_10k',
 'HD01_VD17': 'greaterthan_200k',
 'HD01_VD03': '10k-14k',
 'HD01_VD04': '15k-19k',
 'HD01_VD05': '20k-24k',
 'HD01_VD06': '25k-29k',
 'HD01_VD07': '30k-34k',
 'HD01_VD08': '35k-39k',
 'HD01_VD09': '40k-44k',
 'HD01_VD10': '45k-49k',
 'HD01_VD11': '50k-59k',
 'HD01_VD12': '60k-74k',
 'HD01_VD13': '75k-99k',
 'HD01_VD14': '100k-124k',
 'HD01_VD15': '125k-149k',
 'HD01_VD16': '150k-199k'}

Now we can rename our columns

In [17]:
df.rename(columns=columnnames,inplace=True)
df.head()

,blockgroup,pop_tot,lessthan_10k,10k-14k,15k-19k,20k-24k,25k-29k,30k-34k,35k-39k,40k-44k,45k-49k,50k-59k,60k-74k,75k-99k,100k-124k,125k-149k,150k-199k,greaterthan_200k
1,480291101001,330,67,19,8,6,24,78,25,10,0,21,15,43,0,0,14,0
2,480291101002,475,59,39,16,22,0,47,8,41,55,56,22,45,8,22,0,35
3,480291101003,948,87,146,15,33,36,59,65,25,41,37,76,146,75,33,14,60
4,480291103001,851,162,149,37,77,21,25,0,0,0,81,75,97,24,22,34,47
5,480291103002,403,49,53,41,0,0,13,53,52,25,56,21,23,0,9,8,0


In [18]:
# Make Columns Ints
for col in df.columns:
    df[col] = df[col].astype('int')
    
#Create Percentage Columns
for col in df.columns[2:]:
    df[col+'_p'] = df[col]/df['pop_tot']

In [19]:
df.columns[3]

'10k-14k'

In [20]:
# Drop columns with less than 100 people
df[df['pop_tot'] > 99]

,blockgroup,pop_tot,lessthan_10k,10k-14k,15k-19k,20k-24k,25k-29k,30k-34k,35k-39k,40k-44k,...,35k-39k_p,40k-44k_p,45k-49k_p,50k-59k_p,60k-74k_p,75k-99k_p,100k-124k_p,125k-149k_p,150k-199k_p,greaterthan_200k_p
1,480291101001,330,67,19,8,6,24,78,25,10,...,0.075758,0.030303,0.000000,0.063636,0.045455,0.130303,0.000000,0.000000,0.042424,0.000000
2,480291101002,475,59,39,16,22,0,47,8,41,...,0.016842,0.086316,0.115789,0.117895,0.046316,0.094737,0.016842,0.046316,0.000000,0.073684
3,480291101003,948,87,146,15,33,36,59,65,25,...,0.068565,0.026371,0.043249,0.039030,0.080169,0.154008,0.079114,0.034810,0.014768,0.063291
4,480291103001,851,162,149,37,77,21,25,0,0,...,0.000000,0.000000,0.000000,0.095182,0.088132,0.113984,0.028202,0.025852,0.039953,0.055229
5,480291103002,403,49,53,41,0,0,13,53,52,...,0.131514,0.129032,0.062035,0.138958,0.052109,0.057072,0.000000,0.022333,0.019851,0.000000
6,480291103003,162,19,22,15,0,6,10,30,7,...,0.185185,0.043210,0.000000,0.067901,0.000000,0.043210,0.043210,0.030864,0.080247,0.061728
7,480291105001,745,321,94,61,57,44,9,18,51,...,0.024161,0.068456,0.018792,0.021477,0.000000,0.057718,0.016107,0.006711,0.000000,0.000000
8,480291106001,203,34,8,24,21,0,0,6,46,...,0.029557,0.226601,0.000000,0.000000,0.315271,0.000000,0.000000,0.000000,0.000000,0.000000
9,480291106002,652,247,144,85,20,25,65,14,0,...,0.021472,0.000000,0.062883,0.016871,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,480291106003,376,147,83,57,29,29,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.039894,0.000000,0.000000,0.000000


In [21]:
X_cols = list(df.columns[18:])

We now have our X Variables for our initial model. Let's get our Ys

In [22]:
ys = pd.read_csv('Data/FCC_deps.csv')

In [23]:
ys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084 entries, 0 to 1083
Data columns (total 5 columns):
blockgroup    1084 non-null int64
pcat_10x1     1084 non-null int64
bcat_10x1     1084 non-null int64
pcat_all      1084 non-null int64
bcat_all      1084 non-null int64
dtypes: int64(5)
memory usage: 42.4 KB


In [24]:
model_df = pd.merge(df,ys).dropna()


for col in model_df.columns:
    print(col)
    print(model_df[col].isnull().sum())
    print()

blockgroup
0

pop_tot
0

lessthan_10k
0

10k-14k
0

15k-19k
0

20k-24k
0

25k-29k
0

30k-34k
0

35k-39k
0

40k-44k
0

45k-49k
0

50k-59k
0

60k-74k
0

75k-99k
0

100k-124k
0

125k-149k
0

150k-199k
0

greaterthan_200k
0

lessthan_10k_p
0

10k-14k_p
0

15k-19k_p
0

20k-24k_p
0

25k-29k_p
0

30k-34k_p
0

35k-39k_p
0

40k-44k_p
0

45k-49k_p
0

50k-59k_p
0

60k-74k_p
0

75k-99k_p
0

100k-124k_p
0

125k-149k_p
0

150k-199k_p
0

greaterthan_200k_p
0

pcat_10x1
0

bcat_10x1
0

pcat_all
0

bcat_all
0



In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [26]:
train,test = train_test_split(model_df)
lm1 = LogisticRegression()

In [27]:
lm1.fit(train[X_cols],train['bcat_10x1'])

/home/snorks/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
train['predicted'] = lm1.predict(train[X_cols])

/home/snorks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
accuracy_score(train['predicted'],train['bcat_10x1'])

0.7879161528976573

In [31]:
test['predicted'] = lm1.predict(test[X_cols])

/home/snorks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
confusion_matrix(test['predicted'],test['bcat_10x1'])

array([[ 93,  36],
       [ 21, 121]])

## Feature Engineering for Future models